In [8]:
# data import
import pandas as pd
import numpy as np
import numpy_financial as npf
import plotly.express as px

mitigation = pd.read_csv('cba_inequal_aversion_zero_e307eed12 (1).csv')
#in this scenario we don't account for the 1,8 target
mitigation = mitigation.drop(columns="Unit").set_index(["Variable", "Region"])
#mitigation.select({"Variable": ["regional_emissions", "baseline", "abatement_costs"]}) werkt niet waarom? 

mitigation_emissions = mitigation.loc["regional_emissions"]
mitigation_baseline=  mitigation.loc["baseline"]
mitigation_costs= mitigation.loc["abatement_costs"]
mitigation_population = mitigation.loc["population"]



In [9]:
global_emissions = mitigation_emissions.sum(axis=0)
global_emissions_2020 = mitigation_emissions.sum(axis=0)
global_emissions_2020 = np.asarray(global_emissions_2020)
global_emissions_2020 = global_emissions_2020[0]


#data selection for 2020 
mitigation_emissions_2020 = mitigation_emissions.loc[:,'2020']
#check if this was the right way 

# calculations of the regional allowance for 2020
#regional allowance GF, alleen van 2020 
GF_fractions = pd.DataFrame({
    str(year): mitigation_emissions_2020/global_emissions_2020
    for year in range(2020, 2151, 5)
})
GF2020= GF_fractions * global_emissions
GF2020.to_csv('GF2020.csv')


#check if this was the right way


# Mitigation cost Cap

In [10]:
# Emission allowance gap calculation 

EAG = mitigation_emissions - GF2020

Emission_reduction= mitigation_baseline-mitigation_emissions


#Total mitigation costs per time step 
TMC = mitigation_costs.sum(axis=0)
 

#total emission reduction (does it have to be per region or year?)

GER = Emission_reduction.sum(axis=0)


#Mitigation cost gap 

MCG= EAG*TMC/GER
MCG.to_csv("output\MCG_GF.csv")



# Calculations of the new mititgation costs 


In [ ]:
#notities meeting, berekenen van de nieuwe mitigatie costen mitigatie kosten scenario - MCG, NPV berekening, 
#NPV avoided damages gedeeld door de (NPV?)mitigatie kosten

# Mitigation Costs and NPV 2100

In [11]:
#new mitigation costs 2100

GF_mitigationcosts = mitigation_costs + MCG 
GF_mitigationcosts.to_csv("output\mitig_costs_GF.csv")
GF_mitigationcosts = GF_mitigationcosts.reset_index()
GF_mitigationcosts= GF_mitigationcosts.melt('Region')
GF_mitigationcosts = GF_mitigationcosts[GF_mitigationcosts.variable<'2101']
GF_mitigationcosts = GF_mitigationcosts.pivot(index ='Region', columns='variable')


# avoided damages 
avoided_damages = pd.read_csv('avoided_damages_ineqav.csv')
avoided_damages = avoided_damages.melt('Region')
avoided_damages = avoided_damages[avoided_damages.variable< '2101']

avoided_damages = avoided_damages.pivot(index ='Region', columns='variable')
# avoided_damages


#NPV 

discount_rate= 0.03 

def npv(dataframe):
    NPV_values=pd.Series({
        region: npf.npv(discount_rate, row)
        for region, row in dataframe.iterrows()
    })
    return NPV_values


GF_NPV= npv(GF_mitigationcosts)
AD = npv(avoided_damages)

GF_NPV.to_csv("NPV_GF.csv")

## CBR 
CBR_GF = AD/GF_NPV
CBR_GF = CBR_GF.to_frame().reset_index()
CBR_GF.rename(columns = {'index':'Region'},
          inplace = True )
CBR_GF.columns = ['Region','CBR']

CBR_GF.to_csv("CBR_GF.csv") 


In [12]:
# global CBR 
discount_rate = 0.03

def npv(dataframe):
    NPV_values=pd.Series({
        column: npf.npv(discount_rate, values)
        for column, values in dataframe.iteritems()
    })
    return NPV_values

Mitglobal = GF_mitigationcosts.sum(axis= 0 ).to_frame()
ADglobal = avoided_damages.sum(axis=0).to_frame()

Mitglobal = npv(Mitglobal)
ADglobal = npv(ADglobal)

CBR_global_mit = ADglobal/Mitglobal
CBR_global_mit.to_csv('CBR_global_mit_GF2100.csv')
CBR_global_mit

0    3.673463
dtype: float64

# Mitigation Costs and NPV 2150

In [14]:
#new mitigation costs 2150 

GF_mitigationcosts = mitigation_costs + MCG 
GF_mitigationcosts.to_csv("output\mitig_costs_GF.csv")

# avoided damages 
avoided_damages = pd.read_csv('avoided_damages_ineqav.csv').set_index('Region')


#NPV 

discount_rate= 0.03 

def npv(dataframe):
    NPV_values=pd.Series({
        region: npf.npv(discount_rate, row)
        for region, row in dataframe.iterrows()
    })
    return NPV_values


GF_NPV= npv(GF_mitigationcosts)
AD = npv(avoided_damages)

GF_NPV.to_csv("NPV_GF_2150.csv")

## CBR 
CBR_GF_2150 = AD/GF_NPV
CBR_GF_2150= CBR_GF_2150.reset_index()
CBR_GF_2150.rename(columns = {'index':'Region'},
          inplace = True )
CBR_GF_2150.columns = ['Region','CBR']

CBR_GF_2150.to_csv("output\CBR_GF_2150.csv")

CBR_GF_2150

,Region,CBR
0,CAN,6.292057
1,USA,4.606472
2,MEX,9.203762
3,RCAM,7.766265
4,BRA,5.828844
5,RSAM,6.825731
6,NAF,14.414066
7,WAF,18.711212
8,EAF,21.976918
9,SAF,7.071442


In [7]:
# global CBR 
discount_rate = 0.03

def npv(dataframe):
    NPV_values=pd.Series({
        column: npf.npv(discount_rate, values)
        for column, values in dataframe.iteritems()
    })
    return NPV_values

Mitglobal = GF_mitigationcosts.sum(axis= 0 ).to_frame()
ADglobal = avoided_damages.sum(axis=0).to_frame()

Mitglobal = npv(Mitglobal)
ADglobal = npv(ADglobal)

CBR_global_mit = ADglobal/Mitglobal
CBR_global_mit.to_csv('CBR_global_mit_GF2150.csv')
CBR_global_mit

0    9.554703
dtype: float64